In [1]:
###########################################
# IMPORTS
###########################################
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactoryV3
from perses.annihilation.lambda_protocol import RESTStateV3
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
# REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("Reference")

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=6)
           2	LOAD_GLOBAL(arg=0, lineno=6)
           4	LOAD_METHOD(arg=1, lineno=6)
           6	LOAD_CONST(arg=1, lineno=6)
           8	CALL_METHOD(arg=1, lineno=6)
          10	STORE_FAST(arg=2, lineno=6)
          12	LOAD_FAST(arg=0, lineno=7)
          14	LOAD_CONST(arg=2, lineno=7)
          16	BINARY_SUBSCR(arg=None, lineno=7)
          18	LOAD_FAST(arg=1, lineno=7)
          20	LOAD_CONST(arg=3, lineno=7)
          22	BINARY_SUBSCR(arg=None, lineno=7)
          24	BINARY_MULTIPLY(arg=None, lineno=7)
          26	LOAD_FAST(arg=0, lineno=7)
          28	LOAD_CONST(arg=3, lineno=7)
          30	BINARY_SUBSCR(arg=None, lineno=7)
          32	LOAD_FAST(arg=1, lineno=7)
          34	LOAD_CONST(arg=2, lineno=7)
          36	BINARY_SUBSCR(arg=None, lineno=7)
          38	BINARY_MULTIPLY(arg=None, lineno=7)
          40	BINARY_SUBTRACT(arg=None, lineno=7)
          42	LOAD_FAST(arg=2, lineno=7)
          44	LO

DEBUG:numba.core.byteflow:dispatch pc=74, inst=BINARY_MULTIPLY(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$66binary_subscr.30', '$72binary_subscr.33']
DEBUG:numba.core.byteflow:dispatch pc=76, inst=BINARY_SUBTRACT(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$74binary_multiply.34']
DEBUG:numba.core.byteflow:dispatch pc=78, inst=LOAD_FAST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35']
DEBUG:numba.core.byteflow:dispatch pc=80, inst=LOAD_CONST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36']
DEBUG:numba.core.byteflow:dispatch pc=82, inst=STORE_SUBSCR(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36', '$const80.37']
DEBUG:numba.core.byteflow:dispatch pc=84, inst=LOAD_FAST(arg=0, lineno=9)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=86, inst=LOAD_CONST(arg=4, lineno=9)
DEBUG:numba.core.bytefl

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x2b11fc4d6520>
DEBUG:numba.core.ssa:on stmt: a = arg(0, name=a)
DEBUG:numba.core.ssa:on stmt: b = arg(1, name=b)
DEBUG:numba.core.ssa:on stmt: $2load_global.0 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-rest2v3/lib/python3.9/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $4load_method.1 = getattr(value=$2load_global.0, attr=zeros)
DEBUG:numba.core.ssa:on stmt: $const6.2 = const(int, 3)
DEBUG:numba.core.ssa:on stmt: c = call $4load_method.1($const6.2, func=$4load_method.1, args=[Var($const6.2, coordinate_numba.py:6)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $const14.5 = const(int, 1)
DEBUG:numba.core.ssa:on stmt: $16binary_subscr.6 = static_getitem(value=a, index=1, index_var=$const14.5, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: $const20.8 = const(int, 2)
DEBUG:numba.core.ssa:on stmt:

DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_GLOBAL(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=LOAD_METHOD(arg=1, lineno=14)
DEBUG:numba.core.byteflow:stack ['$2load_global.0']
DEBUG:numba.core.byteflow:dispatch pc=6, inst=LOAD_FAST(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1']
DEBUG:numba.core.byteflow:dispatch pc=8, inst=LOAD_FAST(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1', '$a6.2']
DEBUG:numba.core.byteflow:dispatch pc=10, inst=CALL_METHOD(arg=2, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1', '$a6.2', '$a8.3']
DEBUG:numba.core.byteflow:dispatch pc=12, inst=STORE_FAST(arg=1, lineno=14)
DEBUG:numba.core.byteflow:stack ['$10call_method.4']
D

DEBUG:numba.core.byteflow:dispatch pc=14, inst=UNPACK_SEQUENCE(arg=1, lineno=509)
DEBUG:numba.core.byteflow:stack ['$12load_attr.5']
DEBUG:numba.core.byteflow:dispatch pc=16, inst=STORE_FAST(arg=3, lineno=509)
DEBUG:numba.core.byteflow:stack ['$14unpack_sequence.6']
DEBUG:numba.core.byteflow:dispatch pc=18, inst=LOAD_FAST(arg=2, lineno=510)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=20, inst=LOAD_FAST(arg=3, lineno=510)
DEBUG:numba.core.byteflow:stack ['$m18.8']
DEBUG:numba.core.byteflow:dispatch pc=22, inst=COMPARE_OP(arg=3, lineno=510)
DEBUG:numba.core.byteflow:stack ['$m18.8', '$n20.9']
DEBUG:numba.core.byteflow:dispatch pc=24, inst=POP_JUMP_IF_FALSE(arg=34, lineno=510)
DEBUG:numba.core.byteflow:stack ['$22compare_op.10']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=26, stack=(), blockstack=(), npush=0), Edge(pc=34, stack=(), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=26 nstack_initial=0), State(pc_initial

...

DEBUG:numba.core.ssa:on stmt: $194load_method.1 = getattr(value=$192load_global.0, attr=arccos)
DEBUG:numba.core.ssa:on stmt: phi = call $194load_method.1(cos_phi.3, func=$194load_method.1, args=[Var(cos_phi.3, coordinate_numba.py:135)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $202load_global.4 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-rest2v3/lib/python3.9/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $204load_method.5 = getattr(value=$202load_global.4, attr=dot)
DEBUG:numba.core.ssa:on stmt: $210call_method.8 = call $204load_method.5(a, plane2, func=$204load_method.5, args=[Var(a, coordinate_numba.py:106), Var(plane2, coordinate_numba.py:127)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $const212.9 = const(int, 0)
DEBUG:numba.core.ssa:on stmt: $214compare_op.10 = $210call_method.8 <= $const212.9
DEBUG:numba.core.ssa:on stmt: bool216 = global(bool: <class 'bool'>)
DEBUG:numba.core.ssa:on stmt: $216pred = call bool216($

DEBUG:numba.core.byteflow:dispatch pc=4, inst=LOAD_FAST(arg=1, lineno=106)
DEBUG:numba.core.byteflow:stack ['$angle_position_12.0']
DEBUG:numba.core.byteflow:dispatch pc=6, inst=BINARY_SUBTRACT(arg=None, lineno=1)
DEBUG:numba.core.byteflow:stack ['$angle_position_12.0', '$bond_position_24.1']
DEBUG:numba.core.byteflow:dispatch pc=8, inst=RETURN_VALUE(arg=None, lineno=1)
DEBUG:numba.core.byteflow:stack ['$6binary_subtract.2']
DEBUG:numba.core.byteflow:end state. edges=[]
DEBUG:numba.core.byteflow:-------------------------Prune PHIs-------------------------
DEBUG:numba.core.byteflow:Used_phis: defaultdict(<class 'set'>, {State(pc_initial=0 nstack_initial=0): set()})
DEBUG:numba.core.byteflow:defmap: {}
DEBUG:numba.core.byteflow:phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:keep phismap: {}
DEBUG:numba.core.byteflow:new_out: defaultdict(<class 'dict'>, {})
DEBUG:numba.core.byteflow:-------------

DEBUG:numba.core.interpreter:label 0:
    _32call_function_12_1 = arg(0, name=_32call_function_12_1) ['_32call_function_12_1']
    a_1 = arg(1, name=a_1)                   ['a_1']
    $6binary_true_divide.2 = a_1 / _32call_function_12_1 ['$6binary_true_divide.2', '_32call_function_12_1', 'a_1']
    $8return_value.3 = cast(value=$6binary_true_divide.2) ['$6binary_true_divide.2', '$8return_value.3']
    return $8return_value.3                  ['$8return_value.3']

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x2b11fce9ac10>
DEBUG:numba.core.ssa:on stmt: _32call_function_12_1 = arg(0, name=_32call_function_12_1)
DEBUG:numba.core.ssa:on stmt: a_1 = arg(1, name=a_1)
DEBUG:numba.core.ssa:on stmt: $6binary_true_divide.2 = a_1 / _32call_function_12_1
DEBUG:numba.core.ssa:on stmt: $8return_value.3 = cast(value=$6binary_true_divide.2)
DEBUG:numba.core.ssa:on stmt: return $8return_value.3
DEBUG:numba.core.ssa:def

DEBUG:numba.core.ssa:SSA violators set()
INFO:rdkit:Enabling RDKit 2021.03.5 jupyter extensions


conducting subsequent work with the following platform: CUDA


In [2]:
def compare_energy_components(rest_system, other_system, positions):
    """
    Get energy components of a given system
    """
    platform = configure_platform(platform_name='CUDA')
    print(platform.getName())
    print(platform.getPropertyDefaultValue('CudaPrecision'))
    print(platform.getPropertyDefaultValue('DeterministicForces'))

    # Create thermodynamic state and sampler state for non-rest system
    thermostate_other = ThermodynamicState(system=other_system, temperature=temperature)

    # Create context for non-rest system
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other, platform=platform)
    context_other.setPositions(positions)

    # Get energy components for non-rest system
    components_other = [component for component in compute_potential_components(context_other, beta=beta, platform=platform)]
    
    # Create thermodynamic state for rest_system
    thermostate_rest = ThermodynamicState(system=rest_system, temperature=temperature)

    # Create context for rest system
    integrator_rest = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_rest = thermostate_rest.create_context(integrator_rest, platform=platform)
    context_rest.setPositions(positions)

    # Get energy components for rest system
    components_rest = [component for component in compute_potential_components(context_rest, beta=beta, platform=platform)]

    print(components_other)
    print(components_rest)
        
    
#     # Check that bond, angle, and torsion energies match
#     for other, rest in zip(components_other[:3], components_rest[:3]):
#         assert np.isclose([other[1]], [rest[1]]), f"The energies do not match for the {other[0]}: {other[1]} (other system) vs. {rest[1]} (REST system)"

#     # Check that nonbonded energies match
    nonbonded_other = np.array([component[1] for component in components_other[3:]]).sum()
#     nonbonded_rest = np.array([component[1] for i, component in enumerate(components_rest) if i in [4]]).sum()
    nonbonded_rest = np.array([component[1] for i, component in enumerate(components_rest) if i >= 3]).sum()
    print(nonbonded_other)
    print(nonbonded_rest)
#     assert np.isclose([nonbonded_other], [nonbonded_rest]), f"The energies do not match for the NonbondedForce: {nonbonded_other} (other system) vs. {nonbonded_rest} (REST system)"
    
#     print("Energy bookkeeping was a success!")
    
    


## Alanine dipeptide in vacuum

In [9]:
# Create vanilla system
ala = AlanineDipeptideVacuum()
system = ala.system
positions = ala.positions
topology = ala.topology



In [10]:
# Create REST system
system.removeForce(4)
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]
factory = RESTTopologyFactoryV3(system, topology, rest_region=rest_atoms)
REST_system = factory.REST_system



INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 0
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]


In [11]:
# Compare energy components
context = compare_energy_components(REST_system, system, positions)


conducting subsequent work with the following platform: CUDA
CUDA
mixed
true
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034551762541440684), ('HarmonicAngleForce', 0.6071357953594018), ('PeriodicTorsionForce', 3.2298480734947232), ('NonbondedForce', -39.18692959961769), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.0345517625414457), ('CustomAngleForce', 0.607135795359443), ('CustomTorsionForce', 3.2298523942437525), ('CustomNonbondedForce', -134.404186091246), ('CustomNonbondedForce', 4.716826483346038), ('CustomBondForce', 90.50042737412815), ('NonbondedForce', 0.0), ('AndersenThermostat', 0.0)]
-39.18692959961769
-39.18693223377181
Energy bookkeeping was a success!


## Alanine dipeptide in solvent

In [79]:
## CASE 2: alanine dipeptide in solvent
# Create vanilla system
ala = AlanineDipeptideExplicit()
system = ala.system
system.removeForce(4)
# system.getForce(3).setNonbondedMethod(0)
positions = ala.positions
topology = ala.topology



In [80]:
# Create REST system
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]
factory = RESTTopologyFactoryV3(system, topology, rest_region=rest_atoms, use_dispersion_correction=True)
REST_system = factory.REST_system



INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 3.2893431387452243
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]


In [5]:
REST_system.getForces()

[<openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x10eb59720> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x190951e70> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x19083ccc0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x10ebb63f0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x10ebb6f30> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x10ebb68d0> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x10ebb6060> >]

In [6]:
rest_atoms

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [56]:
# system.getForce(3).setIncludeDirectSpace(False)

In [37]:
system.getForce(3).setReciprocalSpaceForceGroup(31) # Only show direct space energy

In [81]:
for i in range(system.getForce(3).getNumParticles()):
    charge, sigma, epsilon = system.getForce(3).getParticleParameters(i)
#     if i > 21:
#         system.getForce(3).setParticleParameters(i, charge*0, sigma, epsilon*0)
#     else:
    system.getForce(3).setParticleParameters(i, charge*0, sigma, epsilon)


In [82]:
for i in range(system.getForce(3).getNumExceptions()):
    p1, p2, charge, sigma, epsilon = system.getForce(3).getExceptionParameters(i)
#     if p1 > 21 or p2 > 21:
#         system.getForce(3).setExceptionParameters(i, p1, p2, charge*0, sigma, epsilon*0)
#     else:
    system.getForce(3).setExceptionParameters(i, p1, p2, charge*0, sigma, epsilon)
    

In [8]:
# for i in range(REST_system.getForce(3).getNumParticles()):
#     params = REST_system.getForce(3).getParticleParameters(i)
# #     if i > 21:
#     params = list(params)
#     params[-3] = params[-3]*0
#     params[-1] = params[-1]*0
#     REST_system.getForce(3).setParticleParameters(i, params)


In [83]:
for i in range(REST_system.getForce(5).getNumBonds()):
    p1, p2, params = REST_system.getForce(5).getBondParameters(i)
#     if p1 > 21 or p2 > 21:
#         params = list(params)
#     #     params[-3] = params[-3]*0
#         params[-2] = params[-2]*0
#         REST_system.getForce(5).setBondParameters(i, p1, p2, params)
#     else:
    params = list(params)
    params[-4] = params[-4]*0
    params[-1] = params[-1]*0
    REST_system.getForce(5).setBondParameters(i, p1, p2, params)


In [86]:
# Compare energy components
context = compare_energy_components(REST_system, system, positions)


conducting subsequent work with the following platform: CUDA
CUDA
mixed
true
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03455181083230295), ('HarmonicAngleForce', 0.6071348482600101), ('PeriodicTorsionForce', 3.229847881682522), ('NonbondedForce', 1211.5520677729273), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.034551810832195946), ('CustomAngleForce', 0.6071348482604659), ('CustomTorsionForce', 3.22985117992216), ('CustomNonbondedForce', -10097.514476908897), ('CustomNonbondedForce', 1203.138762530719), ('CustomBondForce', 8.413314619996116), ('NonbondedForce', -80578.88533303604), ('AndersenThermostat', 0.0)]
1211.5520677729273
1211.5520771507151


In [35]:
for k, v in context.getParameters().items():
    print(k, v)

AndersenCollisionFrequency 1.0
AndersenTemperature 300.0
lambda_rest_angles 1.0
lambda_rest_bonds 1.0
lambda_rest_electrostatics 1.0
lambda_rest_electrostatics_exceptions 1.0
lambda_rest_sterics 1.0
lambda_rest_sterics_exceptions 1.0
lambda_rest_torsions 1.0


# Create two particle system

In [10]:
from simtk.openmm import app

In [11]:
system = openmm.System()
system.addParticle(22.99)
system.addParticle(35.45)
system.setDefaultPeriodicBoxVectors(openmm.Vec3(9, 0, 0), openmm.Vec3(0, 9, 0), openmm.Vec3(0, 0, 9))

system.addForce(openmm.HarmonicBondForce())
system.addForce(openmm.HarmonicAngleForce())
system.addForce(openmm.PeriodicTorsionForce())

nb_force = openmm.NonbondedForce()
system.addForce(nb_force)
nb_force.setNonbondedMethod(openmm.NonbondedForce.PME)
nb_force.setCutoffDistance(0.3 * unit.nanometer)
nb_force.addParticle(1.0, 0.24, 0.37)
nb_force.addParticle(-1.0, 0.45, 0.15)
positions = [openmm.Vec3(0.1, 0, 0), openmm.Vec3(0.3, 0, 0)]

In [12]:
# Create new topology
topology = app.Topology()

# Copy residues and atoms to new topology for chains_to_keep
chain = topology.addChain(id='A')
res = topology.addResidue('NA', chain, id='1')
atom = topology.addAtom('Na', openmm.app.element.sodium, res)
res = topology.addResidue('CL', chain, id='1')
atom = topology.addAtom('Cl', openmm.app.element.chlorine, res)

In [13]:
# Create REST system
factory = RESTTopologyFactoryV3(system, topology, rest_region=[0], use_dispersion_correction=True)
REST_system = factory.REST_system



INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 8.760869616261553
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=9.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=9.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=9.0), unit=nanometer)]


In [14]:
context = compare_energy_components(REST_system, system, positions)


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0), ('HarmonicAngleForce', 0.0), ('PeriodicTorsionForce', 0.0), ('NonbondedForce', -26.206330051458842), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.0), ('CustomAngleForce', 0.0), ('CustomTorsionForce', 0.0), ('CustomNonbondedForce', -3.6801297504416146), ('CustomNonbondedForce', 252.299897580758), ('CustomBondForce', 0.0), ('NonbondedForce', -274.8260819240421), ('AndersenThermostat', 0.0)]
-26.206330051458842
-26.206314093725723
Energy bookkeeping was a success!


# Create three particle system

In [15]:
from simtk.openmm import app

In [16]:
system = openmm.System()
system.addParticle(22.99)
system.addParticle(35.45)
system.addParticle(35.45)
system.setDefaultPeriodicBoxVectors(openmm.Vec3(1, 0, 0), openmm.Vec3(0, 1, 0), openmm.Vec3(0, 0, 1))

system.addForce(openmm.HarmonicBondForce())
system.addForce(openmm.HarmonicAngleForce())
system.addForce(openmm.PeriodicTorsionForce())

nb_force = openmm.NonbondedForce()
system.addForce(nb_force)
nb_force.setNonbondedMethod(openmm.NonbondedForce.PME)
nb_force.setCutoffDistance(0.3 * unit.nanometer)
nb_force.addParticle(1.0, 0.24, 0.37)
nb_force.addParticle(-0.5, 0.45, 0.15)
nb_force.addParticle(-0.5, 0.45, 0.15)
nb_force.addException(1, 2, 0.5, (0.45+0.45)/2, np.sqrt(0.15*0.15))

positions = [openmm.Vec3(0.3, 0, 0), openmm.Vec3(0.1, 0, 0), openmm.Vec3(0.5, 0, 0)]

In [17]:
# Create new topology
topology = app.Topology()

# Copy residues and atoms to new topology for chains_to_keep
chain = topology.addChain(id='A')
res = topology.addResidue('NA', chain, id='1')
atom = topology.addAtom('Na', openmm.app.element.sodium, res)
res = topology.addResidue('CL', chain, id='2')
atom = topology.addAtom('Cl', openmm.app.element.chlorine, res)
res = topology.addResidue('CL', chain, id='3')
atom = topology.addAtom('Cl', openmm.app.element.chlorine, res)

In [18]:
# Create REST system
factory = RESTTopologyFactoryV3(system, topology, rest_region=[0], use_dispersion_correction=True)
REST_system = factory.REST_system



INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 8.760869616261553
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=1.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=1.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=1.0), unit=nanometer)]


In [19]:
context = compare_energy_components(REST_system, system, positions)


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.0), ('HarmonicAngleForce', 0.0), ('PeriodicTorsionForce', 0.0), ('NonbondedForce', 299.0216087349235), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.0), ('CustomAngleForce', 0.0), ('CustomTorsionForce', 0.0), ('CustomNonbondedForce', -3.6801297504416115), ('CustomNonbondedForce', 506.5023682303346), ('CustomBondForce', 35.31368152781532), ('NonbondedForce', -239.11428040796847), ('AndersenThermostat', 0.0)]
299.0216087349235
299.0216395997398
Energy bookkeeping was a success!


# Run all tests

In [7]:
def test_bookkeeping():
    """
    Given the default Global variables, do energy component bookkeeping with the REST system and the original system.
    Specifically, the first CustomBondForce must match the HarmonicBondForce
    (same with the second and third forces with the HarmonicAngle and PeriodicTorsionForces).
    The sum of the NonbondedForce, the CustomNonbondedForce, and the CustomBondForce (last one) must be equal to the
    energy of the original system's NonbondedForce.
    """

    ## CASE 1: alanine dipeptide in vacuum
    # Create vanilla system
    ala = AlanineDipeptideVacuum()
    system = ala.system
    positions = ala.positions
    topology = ala.topology

    # Create REST system
    system.removeForce(4)
    res1 = list(ala.topology.residues())[1]
    rest_atoms = [atom.index for atom in res1.atoms()]
    factory = RESTTopologyFactoryV3(system, topology, rest_region=rest_atoms)
    REST_system = factory.REST_system

    # Compare energy components
    compare_energy_components(REST_system, system, positions)

    ## CASE 2: alanine dipeptide in solvent
    # Create vanilla system
    ala = AlanineDipeptideExplicit()
    system = ala.system
    positions = ala.positions
    topology = ala.topology

    # Create REST system
    system.removeForce(4)
    res1 = list(ala.topology.residues())[1]
    rest_atoms = [atom.index for atom in res1.atoms()]
    factory = RESTTopologyFactoryV3(system, topology, rest_region=rest_atoms, use_dispersion_correction=True)
    REST_system = factory.REST_system

    # Compare energy components
    compare_energy_components(REST_system, system, positions)

    ## CASE 3: alanine dipeptide in solvent with repartitioned hybrid system
    # Create repartitioned hybrid system for lambda 0 endstate
    atp, system_generator = generate_atp(phase='solvent')
    htf = generate_dipeptide_top_pos_sys(atp.topology,
                                         new_res='THR',
                                         system=atp.system,
                                         positions=atp.positions,
                                         system_generator=system_generator,
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=0,
                                         validate_endstate_energy=False)
    
    # Create REST-ified hybrid system
    res1 = list(htf.hybrid_topology.residues)[1]
    rest_atoms = [atom.index for atom in list(res1.atoms)]
    factory = RESTTopologyFactoryV3(htf.hybrid_system, htf.hybrid_topology, rest_region=rest_atoms, use_dispersion_correction=True)
    REST_system = factory.REST_system

    # Compare energy components
    compare_energy_components(REST_system, htf.hybrid_system, htf.hybrid_positions)


In [8]:
test_bookkeeping()

INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 0
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 3.2893431387452243
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 0.03455064996280121), ('HarmonicAngleForce', 0.6071369391574322), ('PeriodicTorsionForce', 3.2298482876455963), ('NonbondedForce', -39.18693169158183), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.03455064996280121), ('CustomAngleForce', 0.6071369391574319), ('CustomTorsionForce', 3.2298482876455954), ('CustomNonbondedForce', -134.40418350398502), ('CustomNonbondedForce', 4.716823991270118), ('CustomBondForce', 90.50042198031858), ('NonbondedForce', 0.0), ('AndersenThermostat', 0.0)]
-39.18693169158183
-39.18693753239634
Energy bookkeeping was a success!
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 0.03454256736202116), ('Ha

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]


generating htf
making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 14, 18, 13, 17, 16, 19, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching fun

...

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 1.0558269152414181), ('CustomAngleForce', 15.749061874823555), ('CustomTorsionForce', 19.193719397769534), ('CustomBondForce', -59.95290763572909)]
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


INFO:geometry:	reduced angle potential = 3.2058324695160406e-13.
INFO:geometry:	reduced angle potential = 1.2915588460963948e-10.
INFO:geometry:	reduced angle potential = 7.39096069988752e-11.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.
INFO:geometry:total reduced potential before atom placement: 9.116481158706167
INFO:geometry:total reduced energy added from growth 

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.0005201491202417058), ('CustomAngleForce', 0.4511199297896573), ('CustomTorsionForce', 7.25046198458677), ('CustomBondForce', 12.970881401086856)]


INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 2.918423065872431
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.56477354, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.56477354), unit=nanometer)]


generating rest system
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 1.0903955094574882), ('HarmonicAngleForce', 16.356284407942475), ('PeriodicTorsionForce', 35.37057219695222), ('NonbondedForce', -998.326288790699), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
[('CustomBondForce', 1.0903955094574882), ('CustomAngleForce', 16.35628440794247), ('CustomTorsionForce', 35.37057219695221), ('CustomNonbondedForce', -7418.759680689047), ('CustomNonbondedForce', 7032.4635409539505), ('CustomBondForce', 48354.465585379934), ('NonbondedForce', -48966.49698156664), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
-998.326288790699
-998.3275359218023
Energy bookkeeping was a success!


# Test barnase:barstar

In [3]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openeye import oechem
import pickle

In [99]:
solvent_delivery = PointMutationExecutor("../../input/mmc2_barstar.pdb",
                        '1',
                        "42",
                        "ALA",
                        ligand_input="../../input/mmc2_barnase.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True, 
                        conduct_endstate_validation=False,
                        generate_rest_capable_hybrid_topology_factory=True
                       )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 4
INFO:geometry:Atom index prop

conducting subsequent work with the following platform: CUDA


INFO:geometry:setting atoms_with_positions context new positions


conducting subsequent work with the following platform: CUDA


INFO:geometry:There are 4 new atoms
INFO:geometry:	reduced angle potential = 0.00016798278614974654.
INFO:geometry:	reduced angle potential = 1.0363655818693627.
INFO:geometry:	reduced angle potential = 0.20232741656747652.
INFO:geometry:	reduced angle potential = 0.9197309664921618.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 733 bond forces in the no-nonbonded final system
INFO:geometry:	there are 2628 angle forces in the no-nonbonded final system
INFO:geometry:	there are 4939 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.


...

INFO:geometry:total reduced potential before atom placement: 4792.466919261115


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


INFO:geometry:total reduced energy added from growth system: -62.39009632834718
INFO:geometry:final reduced energy 4730.076822938585
INFO:geometry:sum of energies: 4730.076822932769
INFO:geometry:magnitude of difference in the energies: 5.816623627197259e-09
INFO:geometry:Final logp_proposal: -9969.239523541937
INFO:setup:charge diff: 0
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...


added energy components: [('CustomBondForce', 0.40839565535236105), ('CustomAngleForce', 1.6220795142979956), ('CustomTorsionForce', 17.503137529087585), ('CustomBondForce', -81.92370902708512)]


INFO:relative:Flattening torsions of unique new/old at lambda = 0/1
INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.7997), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:

In [100]:
complex_htf = solvent_delivery.get_complex_rhtf_0()
pickle.dump(complex_htf, open("T42A_complex_0.pickle", "wb" ))

apo_htf = solvent_delivery.get_apo_rhtf_0()
pickle.dump(apo_htf, open("T42A_apo_0.pickle", "wb" ))

In [4]:
with open("T42A_complex_0.pickle", "rb") as f:
    htf = pickle.load(f)
    system = htf.hybrid_system
    system.removeForce(0)

# with open("T42A_apo_0.pickle", "rb") as f:
#     htf = pickle.load(f)
#     system = htf.hybrid_system
#     system.removeForce(0)

In [5]:
# Create REST-ified hybrid system
rest_atoms = list(htf._atom_classes['unique_old_atoms']) + list(htf._atom_classes['unique_new_atoms'])
factory = RESTTopologyFactoryV3(system, htf.hybrid_topology, rest_region=rest_atoms, use_dispersion_correction=True)
REST_system = factory.REST_system

INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 2.7569734238004693
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.7997), unit=nanometer)]


In [145]:
# system.getForce(3).setIncludeDirectSpace(False)

In [123]:
system.getForce(3).setReciprocalSpaceForceGroup(31) # Only show direct space energy

In [146]:
for i in range(system.getForce(3).getNumParticles()):
    charge, sigma, epsilon = system.getForce(3).getParticleParameters(i)
#     if i > 21:
#         system.getForce(3).setParticleParameters(i, charge*0, sigma, epsilon*0)
#     else:
    system.getForce(3).setParticleParameters(i, charge*0, sigma, epsilon)


In [147]:
for i in range(system.getForce(3).getNumExceptions()):
    p1, p2, charge, sigma, epsilon = system.getForce(3).getExceptionParameters(i)
#     if p1 > 21 or p2 > 21:
#         system.getForce(3).setExceptionParameters(i, p1, p2, charge*0, sigma, epsilon*0)
#     else:
    system.getForce(3).setExceptionParameters(i, p1, p2, charge*0, sigma, epsilon*0)
    

In [123]:
# for i in range(REST_system.getForce(3).getNumParticles()):
#     params = REST_system.getForce(3).getParticleParameters(i)
# #     if i > 21:
#     params = list(params)
#     params[-3] = params[-3]*0
#     params[-1] = params[-1]*0
#     REST_system.getForce(3).setParticleParameters(i, params)


In [148]:
for i in range(REST_system.getForce(5).getNumBonds()):
    p1, p2, params = REST_system.getForce(5).getBondParameters(i)
#     if p1 > 21 or p2 > 21:
#         params = list(params)
#     #     params[-3] = params[-3]*0
#         params[-2] = params[-2]*0
#         REST_system.getForce(5).setBondParameters(i, p1, p2, params)
#     else:
    params = list(params)
    params[-4] = params[-4]*0
    params[-2] = params[-2]*0
    params[-1] = params[-1]*0
    REST_system.getForce(5).setBondParameters(i, p1, p2, params)


In [6]:
# Compare energy components
compare_energy_components(REST_system, system, htf.hybrid_positions)


conducting subsequent work with the following platform: CUDA
CUDA
mixed
true
conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.42281637373944), ('HarmonicAngleForce', 985.4863168499261), ('PeriodicTorsionForce', 3830.5254286488284), ('NonbondedForce', 1009544.7739251752), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 190.422816373736), ('CustomAngleForce', 985.4863168499243), ('CustomTorsionForce', 3830.525597734782), ('CustomNonbondedForce', -144729.3946850595), ('CustomNonbondedForce', 1164850.2942324295), ('CustomBondForce', 809622.4274020933), ('NonbondedForce', -820199.4379289018), ('AndersenThermostat', 0.0)]
1009544.7739251752
1009543.8890205615


### Check that pme params in context match

In [45]:
system = complex_htf.hybrid_system
platform = configure_platform(platform_name='CUDA')
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context = openmm.Context(system, integrator_other, platform)
system.getForce(4).getPMEParametersInContext(context)

conducting subsequent work with the following platform: CUDA


[2.7569734238004693, 70, 70, 70]

In [46]:
platform = configure_platform(platform_name='CUDA')
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context = openmm.Context(REST_system, integrator_other, platform)
REST_system.getForce(7).getPMEParametersInContext(context)

conducting subsequent work with the following platform: CUDA


[2.7569734238004693, 70, 70, 70]

## Double check that groups = 1 << i is the same as groups = set{[i]}

In [25]:
from openmmtools import utils
DEFAULT_PLATFORM = utils.get_fastest_platform()
def compute_potential_components2(context, beta = beta, platform = DEFAULT_PLATFORM):
    """
    Compute potential energy, raising an exception if it is not finite.
    Parameters
    ----------
    context : simtk.openmm.Context
        The context from which to extract, System, parameters, and positions.
    """
    # Make a deep copy of the system.
    import copy

    from perses.dispersed.utils import configure_platform
    platform = configure_platform(platform.getName(), fallback_platform_name='Reference', precision='double')

    system = context.getSystem()
    system = copy.deepcopy(system)
    # Get positions.
    positions = context.getState(getPositions=True).getPositions(asNumpy=True)
    # Get Parameters
    parameters = context.getParameters()
    # Segregate forces.
    for index in range(system.getNumForces()):
        force = system.getForce(index)
        force.setForceGroup(index)
    # Create new Context.
    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator, platform)
    context.setPositions(positions)
    for (parameter, value) in parameters.items():
        context.setParameter(parameter, value)
    energy_components = list()
    for index in range(system.getNumForces()):
        force = system.getForce(index)
        forcename = force.__class__.__name__
        groups = 1<<index
        potential = beta * context.getState(getEnergy=True, groups=groups).getPotentialEnergy()
        energy_components.append((forcename, potential))
    groups = 1<<31
    potential = beta * context.getState(getEnergy=True, groups=groups).getPotentialEnergy()
    energy_components.append(('reciprocal', potential))
    del context, integrator
    return energy_components

In [22]:
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

In [23]:
## CASE 2: alanine dipeptide in solvent
# Create vanilla system
ala = AlanineDipeptideExplicit()
system = ala.system
system.removeForce(4)
# system.getForce(3).setNonbondedMethod(0)
positions = ala.positions
topology = ala.topology

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(ala.positions)

compute_potential_components2(context_other)

conducting subsequent work with the following platform: CPU


[('HarmonicBondForce', 0.03454256736202116),
 ('HarmonicAngleForce', 0.6071512895607732),
 ('PeriodicTorsionForce', 3.2298499815095068),
 ('NonbondedForce', -9889.983001429402),
 ('AndersenThermostat', 0.0),
 ('reciprocal', 0.0)]

In [26]:
## CASE 2: alanine dipeptide in solvent
# Create vanilla system
ala = AlanineDipeptideExplicit()
system = ala.system
system.removeForce(4)
# system.getForce(3).setNonbondedMethod(0)
positions = ala.positions
topology = ala.topology

system.getForce(3).setReciprocalSpaceForceGroup(31) # Move reciprocal space to group 31

## COMMENT IN THE 2 LINES IN COMPUTE_POTENTIAL_COMPONENTS2 ##

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(ala.positions)

compute_potential_components2(context_other)

conducting subsequent work with the following platform: CPU


[('HarmonicBondForce', 0.03454256736202116),
 ('HarmonicAngleForce', 0.6071512895607732),
 ('PeriodicTorsionForce', 3.2298499815095068),
 ('NonbondedForce', 70688.90357261203),
 ('AndersenThermostat', 0.0),
 ('reciprocal', -80578.88657404146)]

The reciprocal and nonbonded force energies here, when added together, match the og nonbonded force energy. And non of the valence energies have changed. Therefore, groups=1 << i is the same as groups = set{[i]} and the way im setting the reciprocal space group is correct.